In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import tensorflow as tf
import seaborn as sns


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/data for raml/Euler_type/raml_train_fea.xlsx')
data

In [ ]:
X_train = data.iloc[:, :-1]
X_train = X_train.astype(float)
X_train

In [ ]:
y_train = pd.read_excel('/content/drive/MyDrive/data for raml/Euler_type/raml_train_label.xlsx')
y_train

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Print the shapes of the datasets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)


In [ ]:
data1 = pd.read_excel('/content/drive/MyDrive/data for raml/Euler_type/raml_val_fea.xlsx')
data1

In [ ]:
X_test = data1.iloc[:, :-1]
X_test = X_test.astype(float)

X_test

In [ ]:
y_test = pd.read_excel('/content/drive/MyDrive/data for raml/Euler_type/raml_val_label.xlsx')
y_test

In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Concatenate, Flatten, Dense

def inception_net_classification(input_shape):       ## Inception-like model

    inputs = Input(input_shape)

    # Convolutional block 1
    conv1_1 = Conv1D(8, 1, activation='relu', padding='same')(inputs)
    conv1_2 = Conv1D(16, 1, activation='relu', padding='same')(inputs)
    conv1_3 = Conv1D(32, 1, activation='relu', padding='same')(inputs)
    concat1 = Concatenate(axis=-1)([conv1_1, conv1_2, conv1_3])
    #pool1 = MaxPooling1D(pool_size=2)(concat1)

    # Convolutional block 2
    conv2_1 = Conv1D(64, 1, activation='relu', padding='same')(concat1)
    conv2_2 = Conv1D(128, 3, activation='relu', padding='same')(conv2_1)
    conv2_3 = Conv1D(256, 5, activation='relu', padding='same')(conv2_2)
    concat2 = Concatenate(axis=-1)([conv2_1, conv2_2, conv2_3])
    #pool2 = MaxPooling1D(pool_size=2)(concat2)

    # Convolutional block 3
    conv3_1 = Conv1D(64, 1, activation='relu', padding='same')(concat2)
    conv3_2 = Conv1D(128, 1, activation='relu', padding='same')(conv3_1)
    conv3_3 = Conv1D(256, 1, activation='relu', padding='same')(conv3_2)
    #concat3 = Concatenate(axis=-1)([conv3_1, conv3_2, conv3_3])
    # #pool3 = MaxPooling1D(pool_size=2)(concat3)

    # Flatten
    flatten = Flatten()(conv3_3)

    # Dense layers
    dense1 = Dense(256, activation='relu')(flatten)
    dense2 = Dense(128, activation='relu')(dense1)

    # Output layer
    output = Dense(1, activation='linear')(dense2)

    # Create model
    model = Model(inputs=inputs, outputs=output)

    return model

In [ ]:
def custom_loss(y_true, y_pred):
    mae_loss = tf.keras.losses.mean_absolute_error(y_true, y_pred)
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    total_loss = mae_loss + mse_loss
    return total_loss

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

reg_model = unet_inception(input_shape=(17, 1))

learning_rate = 0.0001
optimizer = Adam(learning_rate=learning_rate)

In [ ]:
reg_model.compile(loss=custom_loss, optimizer= optimizer, metrics=['mae', 'mse'])

In [ ]:
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')

In [ ]:
reg_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[checkpoint])

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the best model
custom_objects = {'custom_loss': custom_loss}
with tf.keras.utils.custom_object_scope(custom_objects):
    # Load the best model
    best_model = load_model('best_model.h5')

# Evaluate on test data
test_results = best_model.evaluate(X_test, y_test)

# Extract MAE and MSE from the test results
test_mae = test_results[1]
test_mse = test_results[2]

print("Test MAE:", test_mae)
print("Test MSE:", test_mse)
